# Building Gradient Agreement Algorithm with MAML

In the last section we saw how Gradient agreement algorithm works. We saw how gradient agreement adds weights to the gradients implying their importance.


Now we will see how to use our gradient agreement algorithm MAML by coding them from scratch with just numpy. For better understanding, we consider a simple binary classification task. We randomly generate our input data and we train them with a simple single layer neural network and try to find the optimal parameter theta. 

Now we will step by step of how exactly we are doing this,

First we import all the necessary libraries,

In [1]:
import numpy as np

## Generate Data Points

Now we define a function called sample_points for generating our input (x,y) pairs. It takes the parameter k as an input which implies number of (x,y) pairs we want to sample. 

In [2]:
def sample_points(k):
    x = np.random.rand(k,50)
    y = np.random.choice([0, 1], size=k, p=[.5, .5]).reshape([-1,1])
    return x,y

The above function returns output as follows, 

In [3]:
x, y = sample_points(10)
print x[0]
print y[0]

[0.86213006 0.85488908 0.73205517 0.43742195 0.13757096 0.89868047
 0.63167011 0.34247553 0.3392788  0.03710747 0.89854446 0.35188684
 0.49703317 0.38018055 0.7631458  0.96567766 0.79066254 0.97301861
 0.92213652 0.76549118 0.51386957 0.12254211 0.02287478 0.54356945
 0.34627381 0.15294395 0.55542084 0.2456739  0.89171505 0.89245352
 0.62508429 0.34307283 0.46678738 0.46477777 0.41564971 0.67397622
 0.46779937 0.12663504 0.04662468 0.45111941 0.05543247 0.16493359
 0.15365002 0.31825373 0.42284395 0.40723585 0.6032624  0.22342533
 0.23117682 0.18153011]
[0]


## Single Layer Neural Network

For simplicity and better understand, we use a neural network with only single layer for predicting the output. i.e,

a = np.matmul(X, theta)

YHat = sigmoid(a)



__*So, we use gradient agreement with MAML for finding this optimal parameter value theta that is generalizable across tasks. So that for a new task, we can learn from a few data points in a lesser time by taking very less gradient steps.*__

## Gradient Agreement with MAML

Now, we define a class called GradientAgreement_MAML where we implement the Gradient Agreement MAML algorithm. In the \__init__  method we will initialize all the necessary variables. Then we define our sigmoid activation function. Followed by we define our train function. 

You can check the comments written above each line of code for understanding.

In [10]:
class GradientAgreement_MAML(object):
    def __init__(self):
        
        #initialize number of tasks i.e number of tasks we need in each batch of tasks
        self.num_tasks = 2
        
        #number of samples i.e number of shots  -number of data points (k) we need to have in each task
        self.num_samples = 10

        #number of epochs i.e training iterations
        self.epochs = 100
        
        #hyperparameter for the inner loop (inner gradient update)
        self.alpha = 0.0001
        
        #hyperparameter for the outer loop (outer gradient update) i.e meta optimization
        self.beta = 0.0001
       
        #randomly initialize our model parameter theta
        self.theta = np.random.normal(size=50).reshape(50, 1)
      
    #define our sigmoid activation function  
    def sigmoid(self,a):
        return 1.0 / (1 + np.exp(-a))
    
    
    #now let us get to the interesting part i.e training :P
    def train(self):
        
        #for the number of epochs,
        for e in range(self.epochs):        
            
            self.theta_ = []
            
            #for storing gradient updates
            self.g = []
            
            #for task i in batch of tasks
            for i in range(self.num_tasks):
               
                #sample k data points and prepare our train set
                XTrain, YTrain = sample_points(self.num_samples)
                
                a = np.matmul(XTrain, self.theta)

                YHat = self.sigmoid(a)

                #since we are performing classification, we use cross entropy loss as our loss function
                loss = ((np.matmul(-YTrain.T, np.log(YHat)) - np.matmul((1 -YTrain.T), np.log(1 - YHat)))/self.num_samples)[0][0]
                
                #minimize the loss by calculating gradients
                gradient = np.matmul(XTrain.T, (YHat - YTrain)) / self.num_samples

                #update the gradients and find the optimal parameter theta' for each of tasks
                self.theta_.append(self.theta - self.alpha*gradient)
                
                #compute the gradient update
                self.g.append(self.theta-self.theta_[i])
                
                               
           #now we calculate the weights
           #we know that weight is the sum of dot product of g_i and g_j divided by a normalization factor. 
            
            normalization_factor = 0
            
            for i in range(self.num_tasks):
                for j in range(self.num_tasks):      
                    normalization_factor += np.abs(np.dot(self.g[i].T, self.g[j]))
                    
            w = np.zeros(self.num_tasks)
            
            for i in range(self.num_tasks):

                for j in range(self.num_tasks):
                    w[i] += np.dot(self.g[i].T, self.g[j])

                w[i] = w[i] / normalization_factor
                
                
     
            #initialize meta gradients
            weighted_gradient = np.zeros(self.theta.shape)
                        
            for i in range(self.num_tasks):
            
                #sample k data points and prepare our test set for meta training
                XTest, YTest = sample_points(10)

                #predict the value of y
                a = np.matmul(XTest, self.theta_[i])
                
                YPred = self.sigmoid(a)
                           
                #compute meta gradients
                meta_gradient = np.matmul(XTest.T, (YPred - YTest)) / self.num_samples
                
                
                weighted_gradient += np.sum(w[i]*meta_gradient)

  
            #update our randomly initialized model parameter theta with the meta gradients
            self.theta = self.theta-self.beta*weighted_gradient/self.num_tasks
                                       
            if e%10==0:
                print "Epoch {}: Loss {}\n".format(e,loss)             
                print 'Updated Model Parameter Theta\n'
                print 'Sampling Next Batch of Tasks \n'
                print '---------------------------------\n'

In [11]:
model = GradientAgreement_MAML()

In [12]:
model.train()

Epoch 0: Loss 2.31910705563

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 10: Loss 5.15767994967

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 20: Loss 2.73213692474

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 30: Loss 3.02219229143

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 40: Loss 1.94991368208

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 50: Loss 3.51899454773

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 60: Loss 3.82146653722

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 70: Loss 1.52969798833

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

--------------------